In [1]:
%%bash

# Instal Java
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [10]:
from os.path import abspath
warehouse_location = abspath('../content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/spark-warehouse')

In [29]:
spark = (
    SparkSession
    .builder
    .config("spark.sql.warehouse.dir",warehouse_location)
    .enableHiveSupport()
    .getOrCreate()
)

# Class 01 - **Introduction to Spark-SQL**

## **Overview of Spark-SQL**

Spark SQL is a mechanism of manipulation of data on Spark that uses a SQL language. 

- Performing ANSI-SQL and HiveQL queries that act directly on the RDDs.
- Creating databases, tables and views, similar to a relational database.
- Data management using the Spark metadata catalog and its integration with the Hive metastore. 

&nbsp;

**Limitations**

Spark is designed to work as online analytical processing (OLAP), not as online transaction processing (OLTP). In other words Spark does not exhibit functions like, insert, alter, delete data in a Database.

# Class 02 - **Databases and Datacatalog**

In [12]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

## **Setup**

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
imdb_path = '/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/title_basics.csv'

In [6]:
options_dict = {
    'sep' : '\t' , 
    'header' : 'True'
}

df_titles = (
      spark.read
    .format('csv')
    .options(**options_dict)  
    .load(imdb_path)
)

df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

## **Databases and Catalog**

The metadata catalog can be accessed by the object

```SparkSession.catalog```

The main functionalities are :

- ```listDatabases()``` : list all available databases;
- ```listTables()``` : list all available tables in a certain database;
- ```listFunctions()``` : list available functions in a certain database;
- ```refreshTable()``` : update the metadata of a given table;
- ```uncacheTable()``` : removes a table saved in memory; 
- ```clearCache()``` : removes all tables saved in memory;

In [ ]:
spark.catalog

In [ ]:
spark.catalog.clearCache()

In [ ]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/content/spark-warehouse')]

In [ ]:
spark.catalog.listFunctions()

[Function(name='!', description=None, className='org.apache.spark.sql.catalyst.expressions.Not', isTemporary=True),
 Function(name='%', description=None, className='org.apache.spark.sql.catalyst.expressions.Remainder', isTemporary=True),
 Function(name='&', description=None, className='org.apache.spark.sql.catalyst.expressions.BitwiseAnd', isTemporary=True),
 Function(name='*', description=None, className='org.apache.spark.sql.catalyst.expressions.Multiply', isTemporary=True),
 Function(name='+', description=None, className='org.apache.spark.sql.catalyst.expressions.Add', isTemporary=True),
 Function(name='-', description=None, className='org.apache.spark.sql.catalyst.expressions.Subtract', isTemporary=True),
 Function(name='/', description=None, className='org.apache.spark.sql.catalyst.expressions.Divide', isTemporary=True),
 Function(name='<', description=None, className='org.apache.spark.sql.catalyst.expressions.LessThan', isTemporary=True),
 Function(name='<=', description=None, cl

In [ ]:
spark.catalog.listTables()

[]

Spark databases are a tool to organize tables. They can be seen as something very close to server databases in relataional databases. Spark uses a default database, which is used to create tables, views and perform queries. The structures persist in different sections: i user change the database, all tabels will remain in the previous database and will need to be required differently. 

Some SQL command to work with databases: 

- `SHOW DATABASES` : list all databases
- `CREATE DATABASE <db_name>` : create database
- `USE <db_name>`: define the database as current database for performing queries.
  - **OBS:** if you change database, is possible to access the previous database by using the prefix 'db_name'. For example :

  > `USE db2` \
  > `SELECT * FROM db1.table` 

- `SELECT current_database()` : returns current database
- `DROP DATABASE IF EXISTS <db_name>` : delete database among those that have been defined. 

# Class 03 - Creating Tables and Views

## Tables

- **Managed Tables** : Spark manages data and metadata of table. Instruction like as DROP TABKE affect the data written in disk.
- **Unmanaged Tables** : Spark manages only metadata. The data written in disk can not be altered.  

In [30]:
df_titles_sample = df_titles.sample(fraction=0.1)

In [17]:
df_titles_sample.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|  Animation,Short|
|tt0000006|    short|   Chinese Opium Den|   Chinese Opium Den|      0|     1894|     \N|             1|            Short|
|tt0000017|    short|Italienischer Bau...|Italienischer Bau...|      0|     1895|     \N|             1|Documentary,Short|
|tt0000024|    short|Opening of the Ki...|Opening of the Ki...|      0|     1895|     \N|            \N|       News,Short|
|tt0000032|    short|Die Serpentintänz...|Die Serpentintänz...|      0|     1895|     \N|             1|            Short|
|tt0000035|    s

### Creating Managed Tables

In [28]:
df_titles_sample.write.saveAsTable("title_basics_managed2")

`CREATE TABLE title_basics_managed (schema)`

### Creating Unmanaged Tables

In [20]:
df_titles_sample.write.option('path','../content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/title_basics_unmanaged').saveAsTable('title_basics_unmanaged')

`CREATE EXTERNAL TABLE title_basics_unmanaged (schema)` \
`USING parquet OPTIONS (path '../content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/title_basics_unmanaged')` 

In [21]:
spark.catalog.listTables()

[Table(name='title_basics_managed', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='title_basics_unmanaged', database='default', description=None, tableType='EXTERNAL', isTemporary=False)]

## View

### Creating Views

In [53]:
df_titles_sample.createOrReplaceTempView('title_basics_view')

`CREATE OR REPLACE TEMP VIEW AS title_basics_view` \
`SELECT * FROM <table_name>`

### Creating Global Views

In [54]:
df_titles_sample.createOrReplaceGlobalTempView('title_basics_global_view')

`CREATE OR REPLACE GLOBAL TEMP VIEW AS title_basics_view` \
`SELECT * FROM <table_name>`

In [24]:
spark.catalog.listTables()

[Table(name='title_basics_managed', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='title_basics_unmanaged', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='title_basics_view', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

### Deleting Views

In [25]:
spark.catalog.dropTempView('title_basics_view')

In [26]:
spark.catalog.dropTempView('title_basics_global_view')

In [27]:
spark.catalog.listTables()

[Table(name='title_basics_managed', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='title_basics_unmanaged', database='default', description=None, tableType='EXTERNAL', isTemporary=False)]

# Class 04 - Queries in Spark-SQL

In [32]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [33]:
spark.sql('SHOW TABLES').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|title_basics_managed|      false|
|  default|title_basics_mana...|      false|
|  default|title_basics_unma...|      false|
+---------+--------------------+-----------+



In [35]:
spark.sql('SELECT * FROM default.title_basics_managed LIMIT 10').show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|  Animation,Short|
|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|Documentary,Short|
|tt0000017|    short|Italienischer Bau...|Italienischer Bau...|      0|     1895|     \N|             1|Documentary,Short|
|tt0000025|    short|The Oxford and Ca...|The Oxford and Ca...|      0|     1895|     \N|            \N| News,Short,Sport|
|tt0000027|    short|Cordeliers' Squar...|Place des Cordeli...|      0|     1895|     \N|             1|Documentary,Short|
|tt0000046|    s

In [46]:
spark.sql("""
CREATE TABLE tab1
(tconst STRING,
titleType STRING,
primaryTitle STRING, 
originalTitle STRING, 
isAdult STRING, 
startYear STRING, 
endYear STRING, 
runtimeMinutes STRING, 
genres STRING)
  """).show()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 2))



AnalysisException: ignored

In [47]:
spark.sql('SELECT * FROM default.title_basics_managed LIMIT 10').show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|  Animation,Short|
|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|Documentary,Short|
|tt0000017|    short|Italienischer Bau...|Italienischer Bau...|      0|     1895|     \N|             1|Documentary,Short|
|tt0000025|    short|The Oxford and Ca...|The Oxford and Ca...|      0|     1895|     \N|            \N| News,Short,Sport|
|tt0000027|    short|Cordeliers' Squar...|Place des Cordeli...|      0|     1895|     \N|             1|Documentary,Short|
|tt0000046|    s

In [48]:
spark.sql('DROP TABLE title_basics_managed').show()

++
||
++
++



In [49]:
spark.sql('SHOW TABLES').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|title_basics_mana...|      false|
|  default|title_basics_unma...|      false|
+---------+--------------------+-----------+



In [55]:
spark.sql('SHOW VIEWS').show()

+---------+-----------------+-----------+
|namespace|         viewName|isTemporary|
+---------+-----------------+-----------+
|         |title_basics_view|       true|
+---------+-----------------+-----------+



In [56]:
df_titles_sample.createOrReplaceTempView('title_basics_view')
spark.sql('SELECT * FROM title_basics_view LIMIT 10').show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|Documentary,Short|
|tt0000029|    short|         Baby's Meal|       Repas de bébé|      0|     1895|     \N|             1|Documentary,Short|
|tt0000030|    short|  Rough Sea at Dover|  Rough Sea at Dover|      0|     1895|     \N|             1|Documentary,Short|
|tt0000043|    short|   Batteuse à vapeur|   Batteuse à vapeur|      0|     1896|     \N|            \N|Documentary,Short|
|tt0000044|    short|          Le bivouac|          Le bivouac|      0|     1896|     \N|             1|            Short|
|tt0000049|    s

In [57]:
spark.sql("SELECT CAST(runtimeMinutes as INT) FROM title_basics_view").show(5)

+--------------+
|runtimeMinutes|
+--------------+
|             1|
|             1|
|             1|
|          null|
|             1|
+--------------+
only showing top 5 rows



In [58]:
spark.sql("SELECT CAST(runtimeMinutes as INT) FROM title_basics_view")\
.withColumn('sum(Min)', f.col('runtimeMinutes') + 1).show(5)

+--------------+--------+
|runtimeMinutes|sum(Min)|
+--------------+--------+
|             1|       2|
|             1|       2|
|             1|       2|
|          null|    null|
|             1|       2|
+--------------+--------+
only showing top 5 rows

